# 卷积神经网络

Convolutional Neural Network。

在使用卷积之前，人们尝试了很多人工神经网络来处理图像问题，但是人工神经网络的参数量非常大，从而导致训练难度非常大。因此，计算机视觉的研究一直停滞不前，难以突破。

直到卷积神经网络的出现，它的两个优秀特点：*稀疏连接*与*平移不变性*，让计算机视觉的研究取得了长足的进步。稀疏连接可以让学习的参数变得很少，而平移不变性则不关心物体出现在图像中什么位置。

## 最简单的卷积

卷积核是什么？如下图中红色的示例，其中的卷积核是最简单的情况，只有一个通道。

步长是什么？卷积上下左右滑动的长度，称为步长，用 stride 表示。根据问题的不同，会取不同的步长，但通常来说步长为 1 或 2。

如下，输入是一个 4x4 的特征图，卷积核的大小为 2x2。

输入特征与卷积核计算时，计算方式是卷积核与输入特征按位做乘积运算然后再求和，其结果为输出特征图的一个元素，下图为计算输出特征图第一个元素的计算方式：

<img src="https://static001.geekbang.org/resource/image/78/20/787c8b346de00dayyd7e2d3504c33320.jpg?wh=1561x891" width="40%" />

完成了第一个元素的计算，接着按从左向右，从上至下的顺序进行滑动卷积核，分别与输入的特征图进行计算，向右侧滑动一个单元的计算方式：

<img src="https://static001.geekbang.org/resource/image/5y/b5/5yy249d2f1221e21a1bdc7d8756f4fb5.jpg?wh=1544x862" width="40%" />

。。。
第一行计算完毕之后，卷积核会回到行首，然后向下滑动一个单元，再重复以上从左至右的滑动计算。

<img src="https://static001.geekbang.org/resource/image/cf/bb/cf4aa3yy8ac31b06f153f9090d3bcebb.jpg?wh=1552x929" width="40%" />


## 标准的卷积

现在将最简单的卷积计算方式延伸到标准的卷积计算方式。

假定，输入的特征有 m 个通道，宽为 w，高为 h；输出有 n 个特征图，宽为 w′，高为 h′；卷积核的大小为 k * k。

这个操作的卷积是什么样子的？
1. 输出特征图的通道数由卷积核的个数决定 ==>卷积核的个数 n。
2. 根据卷积计算的定义，输入特征图有 m 个通道 ==>每个卷积核里要也要有 m 个通道。

所以，需要 n 个卷积核，每个卷积核的大小为 (m, k, k)。

<img src="https://static001.geekbang.org/resource/image/62/12/62fd6c269cee17e778f8d5acf085be12.jpeg?wh=1920x1080" width="55%" />

结合上面的图解可以看到，
- 卷积核 1 与全部输入特征进行卷积计算，就获得了输出特征图中第 1 个通道的数据
- 卷积核 2 与全部输入特征图进行计算获得输出特征图中第 2 个通道的数据。
- 以此类推，最终就能计算 n 个输出特征图。
- 现在有 m 个通道，输入特征的每一个通道与卷积核中对应通道的数据按上面的方式进行卷积计算，也就是输入特征图中第 i 个特征图与卷积核中的第 i 个通道的数据进行卷积。这样计算后会生成 m 个特征图，然后将这 m 个特征图按对应位置求和即可，求和后 m 个特征图合并为输出特征中一个通道的特征图。

用公式表示当输入有多个通道时，每个卷积核是如何与输入进行计算的：
- Outputi​ 表示计算第 i 个输出特征图，i 的取值为 1 到 n；
- kernelk​ 表示 1 个卷积核里的第 k 个通道的数据；
- inputk​ 表示输入特征图中的第 k 个通道的数据；
- biasi​ 为偏移项，我们在训练时一般都会默认加上；
- 卷积计算：
   <img src="../res/images/torch-cnn-calc.jpg" width="40%" />

   >为什么要加 bias。与回归方程相似，如果不加 bias 的话，回归方程为 y=wx 不管 w 如何变化，回归方程都必须经过原点。如果加上 bias 的话，回归方程变为 y=wx+b，这样就不是必须经过原点，可以变化的更加多样。

### padding

有的时候可能需要对特征图进行补零操作，这样做主要有两个目的：
1. 需要输入与输出的特征图保持一样的大小；
2. 让输入的特征保留更多的信息。

一般什么情况下会希望特征图变得不那么小？

如果不补零且步长（stride）为 1 的情况下，当有多层卷积层时，特征图会一点点变小。如果希望有更多层卷积层来提取更加丰富的信息时，就可以考虑补零让特征图变小的速度稍微慢一些。这个补零的操作就叫做 padding，padding 等于 1 就是补一圈的零，等于 2 就是补两圈的零。

<img src="https://static001.geekbang.org/resource/image/99/08/99dc22a96df665e93e881yy3cf358d08.jpg?wh=1520x792" width="40%" />

在 Pytorch 中，padding 这个参数可以是字符串、int 和 tuple。
现在分别来看看不同参数类型的使用：
- 当为字符串时只能取 `valid` 与 `same`。
- 当给定整型时，则是要在特征图外边补多少圈 0。
- 给定 tuple 时，则表示在特征图的行与列分别指定补多少零。

**padding 设置为 `same`**

再来看上面卷积计算的例子，

当滑动到特征图最右侧时，发现输出的特征图的宽与输入的特征图的宽不一致，它会自动补零，直到输出特征图的宽与输入特征图的宽一致为止。如下图所示：

<img src="https://static001.geekbang.org/resource/image/52/c7/52f8d0ba39b49e9a2736c1a0afb38cc7.jpg?wh=1463x720" width="40%" />

同理，当计算到特征图的底部时，发现输出特征图的高与输入特征图的高不一致时，它同样会自动补零，直到输入和输出一致为止，如下图所示：

<img src="https://static001.geekbang.org/resource/image/c4/81/c48614da6c7bbcd5abdaf942ea45b481.jpg?wh=1476x735" width="40%" />

完成上述操作，就可以获得与输入特征图有相同高、宽的输出特征图了。

## PyTorch 中的卷积

卷积操作定义在 torch.nn 模块中，这个模块提供了很多构建网络的基础层与方法，如， nn.Conv1d、nn.Conv2d 与 nn.Conv3d 。
nn.Conv2d 使用得最多，而 nn.Conv1d 与 nn.Conv3d 只是输入特征图的维度有所不一样而已，很少会被用到。

```python
class torch.nn.Conv2d(in_channels, 
                      out_channels, 
                      kernel_size, 
                      stride=1, 
                      padding=0, 
                      dilation=1, 
                      groups=1, 
                      bias=True, 
                      padding_mode='zeros', 
                      device=None, 
                      dtype=None)
```

1. in_channels 是指输入特征图的通道数，数据类型为 int。如上标准卷积中 in_channels 为 m；
1. out_channels 是输出特征图的通道数，数据类型为 int。如上标准卷积中 out_channels 为 n。
1. kernel_size 是卷积核的大小，数据类型为 int 或 tuple，需要注意的是只给定卷积核的高与宽即可。如上标准卷积中 kernel_size 为 k。
1. stride 为滑动的步长，数据类型为 int 或 tuple，默认是 1，在前面的例子中步长都为 1。
1. padding 为补零的方式，注意当 padding 为`valid` 与 `same`时，stride 必须为 1。
1. bias 是否使用偏移项。
1. dilation 空洞率。
1. groups 将输入特征图分组计算。深度可分离卷积中，该值等于输入特征图通道数。

>对于 kernel_size、stride、padding 都可以是 tuple 类型，当为 tuple 类型时，第一个维度用于 height 的信息，第二个维度时用于 width 的信息。
>
>卷积背后的理论比较复杂，但在 PyTorch 中实现却很简单。在卷积计算中涉及的几大要素：输入通道数、输出通道数、步长、padding、卷积核的大小，分别对应的就是 nn.Conv2d 的关键参数。所以，要熟练用好 nn.Conv2d()。


### 验证 same 方式

In [1]:
import torch
import torch.nn as nn

首先，**创建（4，4，1）大小的输入特征图**

In [2]:
input_feat = torch.tensor([[4, 1, 7, 5], [4, 4, 2, 5], [7, 7, 2, 4], [1, 0, 2, 4]], dtype=torch.float32)
print(input_feat)
print(input_feat.shape)

tensor([[4., 1., 7., 5.],
        [4., 4., 2., 5.],
        [7., 7., 2., 4.],
        [1., 0., 2., 4.]])
torch.Size([4, 4])


然后，**创建一个 2x2 的卷积**

In [3]:
# 默认情况的随机初始化
conv2d = nn.Conv2d(1, 1, (2, 2), stride=1, padding='same', bias=True)

print(conv2d.weight)
print(conv2d.bias)

Parameter containing:
tensor([[[[ 0.0193,  0.0743],
          [-0.2032,  0.4715]]]], requires_grad=True)
Parameter containing:
tensor([0.0125], requires_grad=True)


此处，需要*强行干预卷积核的初始化*

*默认情况下是随机初始化的*，不会人工强行干预卷积核的初始化。此处，为了验证 `padding=same`，对卷积核的参数进行干预。代码如下：

In [4]:
conv2d = nn.Conv2d(1, 1, (2, 2), stride=1, padding='same', bias=False)

# 卷积核要有四个维度(输入通道数，输出通道数，高，宽)
kernels = torch.tensor([[[[1, 0], [2, 1]]]], dtype=torch.float32)

conv2d.weight = nn.Parameter(kernels, requires_grad=False)
print(conv2d.weight)
print(conv2d.bias)

Parameter containing:
tensor([[[[1., 0.],
          [2., 1.]]]])
None


最后，完成以上两部，就可以**开始计算**，使用已经准备好的输入数据与卷积数据计算得到输出，代码如下：

In [ ]:
output = conv2d(input_feat)

# RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [4, 4]

这里报错了，提示信息是输入的特征图需要是一个 4 维的，而给定的输入特征图是一个 4x4 的 2 维特征图。这是为什么呢？

Pytorch 输入 tensor 的维度信息是 (batch_size, 通道数，高，宽)，但上面只给定了高与宽，没有给定 batch_size 与通道数。所以，回到第一步将输入的 tensor 改为 (1,1,4,4) 的形式。

>在训练时，不会将所有数据一次性加载进来训练，而是以多个批次进行读取的，每次读取的量成为 batch_size

In [6]:
# input_feat = torch.tensor([[4, 1, 7, 5], [4, 4, 2, 5], [7, 7, 2, 4], [1, 0, 2, 4]], dtype=torch.float32)
print(input_feat)

input_feat = input_feat.unsqueeze(0).unsqueeze(0)
print(input_feat)
print(input_feat.shape)


tensor([[4., 1., 7., 5.],
        [4., 4., 2., 5.],
        [7., 7., 2., 4.],
        [1., 0., 2., 4.]])
tensor([[[[4., 1., 7., 5.],
          [4., 4., 2., 5.],
          [7., 7., 2., 4.],
          [1., 0., 2., 4.]]]])
torch.Size([1, 1, 4, 4])


In [8]:
output = conv2d(input_feat)

# 查看输出特征图
output

tensor([[[[16., 11., 16., 15.],
          [25., 20., 10., 13.],
          [ 9.,  9., 10., 12.],
          [ 1.,  0.,  2.,  4.]]]])

## 深度可分离卷积（Depthwise Separable Convolution）

随着深度学习技术的不断发展，许多很深、很宽的网络模型被提出，例如，VGG、ResNet、SENet、DenseNet 等，这些网络利用其复杂的结构，可以更加精确地提取出有用的信息。同时也伴随着硬件算力的不断增强，可以将这些复杂的模型直接部署在服务器端，*在工业中可以落地的项目*中都取得了非常优秀的效果。

但这些模型具有一个通病，就是*速度较慢*、*参数量大*，这两个问题使得这些模型*无法被直接部署到移动终端*。因此，很多研究将目光投入到寻求更加轻量化的模型当中，*这些轻量化模型的要求是速度快、体积小，精度上允许比服务器端的模型稍微降低一些*。

深度可分离卷积就是在这种情况下提出的。

深度可分离卷积由 Depthwise（DW）和 Pointwise（PW）两部分卷积组合而成的。

**Depthwise（DW）卷积**

DW 卷积就是有 m 个卷积核的卷积，每个卷积核中的通道数为 1，这 m 个卷积核分别与输入特征图对应的通道数据做卷积运算，所以 DW 卷积的输出是有 m 个通道的特征图。通常来说，DW 卷积核的大小是 3x3 的。

<img src="https://static001.geekbang.org/resource/image/6b/e9/6baeb5b36cea5c04555ea6186de577e9.jpeg?wh=1920x1080" width="60%" />

**Pointwise（PW）卷积**

DW 忽略了输入特征图通道之间的信息，因此在 DW 之后还要加一个 PW 卷积。PW 卷积也叫做逐点卷积。PW 卷积的主要作用就是将 DW 输出的 m 个特征图结合在一起考虑，再输出一个具有 n 个通道的特征图。

在卷积神经网络中，经常可以看到使用 1x1 的卷积，1x1 的卷积主要作用就是升维与降维。所以，*在 DW 的输出之后的 PW 卷积，就是 n 个卷积核的 1x1 的卷积，每个卷积核中有 m 个通道的卷积数据*。

<img src="https://static001.geekbang.org/resource/image/7e/17/7e2c1a874d7b467bc96d4243813f3017.jpeg?wh=1920x1080" width="60%" />

经过 DW 与 PW 的组合，就可以获得一个与标准卷积有同样输出尺寸的轻量化卷积。

**计算量**

有 m 个通道的输入特征图，卷积核尺寸为 k×k，输出特征图的尺寸为 (n,h′,w′)，那么标准的卷积的计算量为：k×k×m×n×h′×w′ 。

如果采用深度可分离卷积，DW 的计算量为：k×k×m×h′×w′，而 PW 的计算量为：1×1×m×n×h′×w′。

标准卷积与深度可分离卷积计算量的比值为：(k×k×m×h′×w′+1×1×m×n×h′×w′) / (k×k×m×n×h′×w′)​= 1/n​+ 1/(k×k)​。

结论：

深度可分离卷积的计算量大约为普通卷积计算量的 1/k^2​。


### PyTorch 中的实现

在 PyTorch 中实现深度可分离卷积，需要分别实现 DW 与 PW 两个卷积。

先看看 DW 卷积，实现 DW 卷积的会用到 nn.Conv2d 中的 groups 参数。groups 参数的作用就是控制输入特征图与输出特征图的分组情况。当 groups 等于 1 的时候，就是标准卷积，而 groups=1 也是 nn.Conv2d 的默认值。
当 groups 不等于 1 的时候，会将输入特征图分成 groups 个组，每个组都有自己对应的卷积核，然后分组卷积，获得的输出特征图也是有 groups 个分组的。需要注意的是，groups 不为 1 的时候，groups 必须能整除 in_channels 和 out_channels。
当 groups 等于 in_channels 时，就是 DW 卷积。

生成一个三通道的 5x5 输入特征图，然后经过深度可分离卷积，输出一个 4 通道的特征图。

DW 卷积，代码如下：

In [9]:
# 生成一个三通道的5x5特征图
input_feat = torch.rand((3, 5, 5)).unsqueeze(0)
print(input_feat.shape)
print(input_feat)

# 请注意 DW 中，输入特征通道数与输出通道数是一样的
in_channels_dw = input_feat.shape[1]
out_channels_dw = input_feat.shape[1]
# 一般来讲 DW 卷积的 kernel size 为 3
kernel_size = 3
stride = 1

# DW 卷积 groups 参数与输入通道数一样
dw = nn.Conv2d(in_channels_dw, out_channels_dw, kernel_size, stride, groups=in_channels_dw)
dw

torch.Size([1, 3, 5, 5])
tensor([[[[0.0338, 0.1245, 0.9464, 0.6416, 0.1698],
          [0.2669, 0.4822, 0.9866, 0.3682, 0.0569],
          [0.3038, 0.3651, 0.7844, 0.6648, 0.9466],
          [0.4460, 0.7335, 0.4767, 0.3050, 0.8031],
          [0.7662, 0.6854, 0.3627, 0.0858, 0.8471]],

         [[0.8520, 0.2522, 0.9444, 0.3099, 0.2886],
          [0.7177, 0.7641, 0.8995, 0.3366, 0.3939],
          [0.2985, 0.4105, 0.0740, 0.6358, 0.4829],
          [0.0051, 0.4297, 0.6057, 0.3689, 0.9114],
          [0.7415, 0.5610, 0.2859, 0.6799, 0.7928]],

         [[0.4386, 0.0867, 0.2525, 0.1525, 0.7620],
          [0.7969, 0.3451, 0.1835, 0.0015, 0.0642],
          [0.4989, 0.4479, 0.3902, 0.5606, 0.4564],
          [0.1120, 0.6919, 0.1221, 0.8388, 0.2389],
          [0.7997, 0.5199, 0.9507, 0.5593, 0.4309]]]])


Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), groups=3)

几个要点：

1. DW 中，输入特征通道数与输出通道数是一样的；
2. 一般来讲，DW 的卷积核为 3x3；
3. DW 卷积的 groups 参数与输出通道数是一样的。

PW 卷积的实现，代码如下：

In [10]:
in_channels_pw = out_channels_dw
out_channels_pw = 4
# PW 卷积的 kernel size为 1
kernel_size_pw = 1

pw = nn.Conv2d(in_channels_pw, out_channels_pw, kernel_size_pw, stride)
pw

Conv2d(3, 4, kernel_size=(1, 1), stride=(1, 1))

In [11]:
output_feat = pw(dw(input_feat))

print(output_feat.shape)
output_feat

torch.Size([1, 4, 3, 3])


tensor([[[[ 0.0475, -0.0499, -0.1248],
          [ 0.0117, -0.1966, -0.2167],
          [-0.3625, -0.0840, -0.1565]],

         [[-0.3750, -0.3843, -0.4598],
          [-0.3879, -0.4610, -0.4698],
          [-0.5589, -0.4220, -0.4749]],

         [[ 0.4806,  0.3767,  0.5214],
          [ 0.4736,  0.3737,  0.4055],
          [ 0.3773,  0.4062,  0.4173]],

         [[ 0.3526,  0.2202,  0.1737],
          [ 0.3113,  0.1330,  0.0754],
          [ 0.0780,  0.2572,  0.2689]]]], grad_fn=<ConvolutionBackward0>)

## 空洞卷积

空洞卷积经常用于图像分割任务当中。图像分割任务的目的是要做到 pixel-wise 的输出，也就是说，对于图片中的每一个像素点，模型都要进行预测。

对于一个图像分割模型，通常会采用多层卷积来提取特征的，随着层数的不断加深，感受野也越来越大。这里有个新名词——“感受野”。

对于图像分割模型有个问题，经过多层的卷积与 pooling 操作之后，特征图会变小。这时需要对较小的特征图进行上采样或反卷积，将特征图扩大到一定尺度，然后再进行预测。

而从一个较小的特征图恢复到一个较大的特征图，这显然会带来一定的信息损失，特别是较小的物体，这种损失是很难恢复的。那问题来了，能不能既保证有比较大的感受野，同时又不用缩小特征图呢？空洞卷积就是解决这个问题的杀手锏，它最大的优点就是不需要缩小特征图，也可以获得更大的感受野。

### 感受野

receptive field 感受野，是计算机视觉领域中经常使用到的一个概念。

伴随着不断的 pooling 或者卷积操作，在卷积神经网络中不同层的特征图是越来越小的。这就意味着在卷积神经网络中，相对于原图来说，*不同层的特征图，其计算区域是不一样的，这个区域就是感受野*。感受野越大，代表着包含的信息更加全面、语义信息更加抽象，而感受野越小，则代表着包含更加细节的语义信息。

>pooling 是卷积神经网络中的一种操作，通常是在一定区域的特征图内取最大值或平均值，用最大值或平均值代替这个区域的所有数据，pooling 操作会使特征图变小。

### 计算方式

回顾标准卷积的计算，蓝图为输入特征图，滑动的阴影为卷积核，绿色的为输出特征图。

<img src="https://static001.geekbang.org/resource/image/77/63/77643b049ac3cd241980b151e0f32063.gif?wh=395x449" width="25%" />

空洞卷积计算示意图：

<img src="https://static001.geekbang.org/resource/image/49/53/4959201e816888c6648f2e78cccfd253.gif?wh=395x381" width="25%" />

与标准卷积的计算方式比较，空洞卷积的计算只不过是将卷积核以一定比例拆分开来。实现起来呢，就是用 0 来充填卷积核。这个分开的比例，一般称之为扩张率，就是 Conv2d 中的 dilation 参数。dilation 参数默认为 1，同样也是可以为 int 或者 tuple。当为 tuple 时，第一位代表行的信息，第二位代表列的信息。

### 小结

深度可分离卷积主要用于轻量化的模型，而空洞卷积主要用于图像分割任务中。

经验：
- 如果需要轻量化模型，让模型变得更小、更快，可以考虑将卷积层替换为深度可分离卷积。
- 如果在做图像分割项目，可以考虑将神经网络靠后的层替换为空洞卷积，看效果是否能有所提高。

### 思考题

随机生成一个 3 通道的 128x128 的特征图，然后创建一个有 10 个卷积核且卷积核尺寸为 3x3（DW 卷积）的深度可分离卷积，对输入数据进行卷积计算。

In [12]:
input_feat = torch.rand((3, 128, 128)).unsqueeze(0)
print('input:', input_feat.shape)

in_channels_dw = input_feat.shape[1]
out_channels_dw = input_feat.shape[1]
kernel_size_dw = 3
stride = 1

dw = nn.Conv2d(in_channels_dw, out_channels_dw, kernel_size_dw, stride, groups=in_channels_dw)

in_channels_pw = out_channels_dw
out_channels_pw = 10
kernel_size_pw = 1
pw = nn.Conv2d(in_channels_pw, out_channels_pw, kernel_size_pw)

output_feat = pw(dw(input_feat))
print('output:', output_feat.shape)

input: torch.Size([1, 3, 128, 128])
output: torch.Size([1, 10, 126, 126])


## 参考

- [Convolution arithmetic](https://github.com/vdumoulin/conv_arithmetic)
